In [34]:
from g4f.client import Client

In [28]:
import json

def generate_prompt(json_input):
    """
    Generates a prompt from a given JSON input.

    Args:
        json_input (str): JSON string containing 'task', 'example', and 'output'.

    Returns:
        str: Generated prompt.
    """
    
    # Load the JSON input
    data = json.loads(json_input)
    
    # Extracting the fields from JSON
    task = data.get('task', '')
    example = data.get('example', '')
    output = data.get('output', '')

    # Constructing the prompt
    prompt = '''
        You are provided with a very important task. Take as many iterations as you need to complete the task. \
        You must follow strict guidelines to ensure the task is completed correctly. \
        
        The guidelines are as follows: 
        ``` 
            1. **Follow the task description:** Always prioritize completing the assigned task as outlined in the initial prompt. 
            2. **Use the example input/output as a guide:**  Base your responses on the provided format and style. 
            3. **Incorporate user-provided examples:**  Learn and adapt from any additional examples the user provides to further refine your understanding of the desired output. 
            4. **Maintain factual accuracy:**  Strive to generate outputs that are consistent with established knowledge and avoid making claims that cannot be substantiated. 
            5. **Prioritize clarity and coherence:**  Ensure your outputs are easy to understand and logically structured. 
            6. **Be creative within the defined parameters:**  While adhering to the task and examples, feel free to use your knowledge to generate informative and engaging content.
            7. **Return only the prompt:**  Your final response should only include the prompt generated based on the user input.
        ```\

        The task: ```Generate a prompt that would take as input 3 variables: 'user_task', 'user_example' and 'user_output'. The prompt must be very descriptive. ```\
        The input example: ``` 
            {
                "task": "HTML to JSON conversion",
                "example": "<h1>This is a heading</h1>\
                            <p>This is a paragraph with some text.</p>\
                            <ul>\
                            <li>Item 1</li>\
                            <li>Item 2</li>\
                            </ul>",
                "output": "{
                                "type": "document",
                                "children": [
                                    { "type": "heading", "text": "This is a heading" },
                                    { "type": "paragraph", "text": "This is a paragraph with some text." },
                                    { "type": "list", "items": ["Item 1", "Item 2"] }
                                ]
                            }"
            }
        ```\
        The output example: ``` 
            **1. Introduction:**\
            I am providing you with an HTML string. Please convert it into a well-structured JSON representation that captures the essential elements and their relationships. Here's an example to illustrate the desired format:\
            
            **2. Example:**\
            
            HTML:\
            ```HTML
            <h1>This is a heading</h1>\
            <p>This is a paragraph with some text.</p>\
            <ul>\
            <li>Item 1</li>\
            <li>Item 2</li>\
            </ul>
            ```\
            
            JSON:\
            ```JSON
            {
                "type": "document",
                "children": [
                    { "type": "heading", "text": "This is a heading" },
                    { "type": "paragraph", "text": "This is a paragraph with some text." },
                    { "type": "list", "items": ["Item 1", "Item 2"] }
                ]
            }
            ```

            **3. Processing Instructions:**\
            1. Parse the HTML string: Analyze the HTML structure and identify elements like headings, paragraphs, lists, etc. and their content.
            2. Create a JSON object: Construct a JSON object with a "type" property representing the overall document type (e.g., "document").
            3. Define child elements: Within the main object, create a "children" array to hold information about child elements.
            4. Map elements to JSON structure: For each element in the HTML, create a corresponding object within the "children" array.
            5. Include a "type" property specifying the element type (e.g., "heading", "paragraph", "list").
            6. Add a "text" property for elements containing text content (e.g., headings, paragraphs).
            7. For lists, include an "items" array containing the list items.
            8. Handle complex elements: While this example covers basic elements, be prepared to handle more complex structures like tables, forms, and nested elements using appropriate JSON representations.

            **4. Other User Examples:**\

        ```
    '''

    prompt += f'''
        The user task: ```{task}```\
        The user example: ```{example}```\
        The user output: ```{output}```\
    '''

    client_local = Client()
    response = client_local.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    return response.choices[0].message.content
    

In [29]:
task_dict = {
    "task": "Transform the unstructured data into structured data.",
    "example": "My neighbor Jhon said that he would be coming to the party. I said that I would be coming too. Then I met my other neighbor Andrew and said that I dont want to see Jhon at the party. Andrew said that he would be coming too.",
    "output": "<Jhon> I would be coming to the party. \n <Me> I would be coming too. [Met Andrew.] <Me> I dont want to see Jhon at the party. <Andrew> I would be coming too."
}

json_input = json.dumps(task_dict)

prompt = generate_prompt(json_input)
print(prompt)


**1. Introduction:**  
I am providing you with a piece of unstructured data. Please transform it into a well-organized and structured format that captures the key information and relationships clearly. Here's an example to illustrate the desired format:

**2. Example:**  
Unstructured Data:  
```
My neighbor Jhon said that he would be coming to the party. I said that I would be coming too. Then I met my other neighbor Andrew and said that I dont want to see Jhon at the party. Andrew said that he would be coming too.
```

Structured Data:  
```
<Jhon> I would be coming to the party. 
<Me> I would be coming too. [Met Andrew.] 
<Me> I dont want to see Jhon at the party. 
<Andrew> I would be coming too.
```

**3. Processing Instructions:**  
1. **Identify Key Entities:** Extract the names of individuals mentioned in the unstructured data (e.g., Jhon, Andrew, etc.).
2. **Segment Statements:** Break down the unstructured text into individual statements made by each entity.
3. **Assign Statem

In [33]:
new_task_examples = {
    "examples": ["My neighbor Jhon said that he would go to army. I said that it is gonna be hard. He said that he would be fine. Then I thought a little. I said him to be careful not to drop the soap."]
}

prompt_with_new_example = prompt
for example in new_task_examples["examples"]:
    prompt_with_new_example += example

client = Client()
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": prompt_with_new_example
        }
    ]
)
print(response.choices[0].message.content)

**Unstructured Data:**  
```
My neighbor Jhon said that he would go to army. I said that it is gonna be hard. He said that he would be fine. Then I thought a little. I said him to be careful not to drop the soap.
```

**Structured Data:**  
```
<Jhon> I would go to the army.
<Me> It is gonna be hard.
<Jhon> I would be fine.
<Me> [Thought a little.] Be careful not to drop the soap.
```

**Processing Steps:**

1. **Identify Key Entities:**  
   - Jhon
   - Me (the speaker)

2. **Segment Statements:**  
   - Jhon: "I would go to army."
   - Me: "It is gonna be hard."
   - Jhon: "I would be fine."
   - Me: "Be careful not to drop the soap."

3. **Assign Statements to Entities:**  
   - `<Jhon> I would go to the army.`
   - `<Me> It is gonna be hard.`
   - `<Jhon> I would be fine.`
   - `<Me> Be careful not to drop the soap.`

4. **Capture Interactions:**  
   - Interaction: "I thought a little."
   - Annotated as: `[Thought a little.]`

5. **Ensure Clarity and Conciseness:**  
   - Organiz